In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer

In [12]:
max_words = 20000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['Text'])
X_sequences = tokenizer.texts_to_sequences(df['Text'])

In [13]:
sequence_len = 10
X = []
y_corpus = []

for i in range(len(X_sequences)):
    num_sequences = len(X_sequences[i]) // sequence_len
    
    for j in range(num_sequences):
        start = sequence_len * j
        end = sequence_len * (j + 1)
        X.append(X_sequences[i][start:end])
        y_corpus.append(df['Category'][i])

In [14]:
X = pd.DataFrame(X)

Y = pd.get_dummies(pd.Series(y_corpus))
Y = Y[ sorted(Y.columns) ]

print(X.shape)
print(Y.shape)

(5115, 10)
(5115, 4)


In [15]:
print(Y.columns)

Index(['Corporate', 'Criminal', 'Divorce', 'Legal_Procedures'], dtype='object')


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(4092, 10) (4092, 4)
(1023, 10) (1023, 4)


In [17]:
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
epochs = 100
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,
                    validation_split=0.1,
#                     callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]
)

Epoch 1/100
58/58 [==============================] - 3s 50ms/step - loss: 1.2141 - accuracy: 0.4715 - val_loss: 0.8146 - val_accuracy: 0.6488
Epoch 2/100
58/58 [==============================] - 2s 42ms/step - loss: 0.4823 - accuracy: 0.8370 - val_loss: 0.4329 - val_accuracy: 0.8512
Epoch 3/100
58/58 [==============================] - 2s 41ms/step - loss: 0.2154 - accuracy: 0.9318 - val_loss: 0.4218 - val_accuracy: 0.8415
Epoch 4/100
58/58 [==============================] - 2s 38ms/step - loss: 0.1308 - accuracy: 0.9620 - val_loss: 0.4828 - val_accuracy: 0.8439
Epoch 5/100
58/58 [==============================] - 2s 42ms/step - loss: 0.0782 - accuracy: 0.9796 - val_loss: 0.5669 - val_accuracy: 0.8390
Epoch 6/100
58/58 [==============================] - 2s 40ms/step - loss: 0.0530 - accuracy: 0.9872 - val_loss: 0.5982 - val_accuracy: 0.8268
Epoch 7/100
58/58 [==============================] - 2s 39ms/step - loss: 0.0382 - accuracy: 0.9908 - val_loss: 0.5976 - val_accuracy: 0.8244
Epoch 

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
model.evaluate(X_test, Y_test)

In [69]:
from tensorflow import keras
import pickle

model.save('saved_model')
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: saved_model\assets


In [125]:
# END

In [ ]:
# plt.title('Loss')
# plt.plot(history.history['loss'], label='train')
# plt.plot(history.history['val_loss'], label='test')
# plt.legend()
# plt.show();

In [ ]:
# from keras.preprocessing.sequence import pad_sequences
# X = pad_sequences(X_sequences, maxlen=sequence_len)
# print(X.shape)

In [ ]:
# sequence_len = 20
# X_corpus = []
# y_corpus = []

# for i in range(len(df['Text'])):
#     text = df['Text'][i].split()
#     num_sequences = len(text) // sequence_len
    
#     for j in range(num_sequences):
#         start = sequence_len * j
#         end = sequence_len * (j + 1)
#         X_corpus.append(' '.join(text[start:end]))
#         y_corpus.append(df['Category'][i])

# from keras.preprocessing.text import Tokenizer
# max_words = 10000
# tokenizer = Tokenizer(num_words=max_words)
# tokenizer.fit_on_texts(X_corpus)
# X_sequences = tokenizer.texts_to_sequences(X_corpus)